<a href="https://colab.research.google.com/github/tkasod2/YS_GC_2025-02_VIS/blob/main/apt_999_cost_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 자료처리용
import pandas as pd
import numpy as np
# scraping용
import requests
import bs4

# 시간기록목적
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 시각화목적
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
import matplotlib
%matplotlib inline

# 시각화 세팅 아래 font_path는 수정필요하여 주석처리함
# font_path = r'C:\\WINDOWS\\Fonts\\malgunsl.ttf'.replace("\\","/")
# font_name = fm.FontProperties(fname=font_path).get_name()
# plt.rc('font', family=font_name)
# plt.rcParams['font.family']

In [5]:
# 시군구코드 매핑을 위해서 dictionary 자료형으로 meta정보 준비
sgg_dict=\
{11110:"종로구",11140:"중구",11170:"용산구",11200:"성동구",11215:"광진구",11230:"동대문구",
11260:"중랑구",11290:"성북구",11305:"강북구",11320:"도봉구",11350:"노원구",11380:"은평구",
11410:"서대문구",11440:"마포구",11470:"양천구",11500:"강서구",11530:"구로구",11545:"금천구",
11560:"영등포구",11590:"동작구",11620:"관악구",11650:"서초구",11680:"강남구",11710:"송파구",
11740:"강동구"}

sgg_codes=list(sgg_dict.keys())

In [6]:
# ---------------------quarter입력 함수
# x1 :연도
# x2 :월일
def fn_quarter(x):
    x1 = x[:4]
    x2 = x[4:]
    if x2 in ["01","02","03"]:
        y2 = "Q1"
    elif x2 in ["04","05","06"]:
        y2 = "Q2"
    elif x2 in ["07","08","09"]:
        y2 = "Q3"
    elif x2 in ["10","11","12"]:
        y2 = "Q4"
    y = x1+y2
    return y
#---------------------연월별 scraping(with API)
# API key는 제 개인거니까 너무 많이 사용하시면 사용량제한걸립니다.
# API 직접 발급 원하시면, data.go.kr 에서 실거래 정보 발급신청을 하신뒤 아래 코드에서
# encoding/ decoding 코드 등을 수정하시면 됩니다.
def fn_ym_parsing(start_yyyymm,end_yyyymm,opt="buy"):
    start = datetime.strptime(str(start_yyyymm), "%Y%m")
    end = datetime.strptime(str(end_yyyymm), "%Y%m")

    # 결과 리스트
    lst_deal_ymd = []

    # start부터 end까지 월 단위로 증가
    while start <= end:
        lst_deal_ymd.append(start.strftime("%Y%m"))
        start += relativedelta(months=1)
    # request settings(API 송수신 목적 key 등을 입력)
    if opt=="buy":
        url=r"http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
    else :
        url=r"http://apis.data.go.kr/1613000/RTMSDataSvcAptRent/getRTMSDataSvcAptRent"
    encoding= r"Fio4KxgNG7DuPMhpyS6PPR%2FKNsNw5BMnkL1aWj%2FumHiOu5dgizBDJmhsMdY7osbCHxIbKnxq%2BW1bJY5TTlBPyw%3D%3D"
    decoding =r"Fio4KxgNG7DuPMhpyS6PPR/KNsNw5BMnkL1aWj/umHiOu5dgizBDJmhsMdY7osbCHxIbKnxq+W1bJY5TTlBPyw=="

    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    # 적재 테이블 생성
    df_saved = pd.DataFrame()

    for deal_ymd in lst_deal_ymd:

        for i in range(len(sgg_codes)):

            sgg_code = str([x for x in sgg_codes][i])
            deal_ymd = deal_ymd

            params ={'serviceKey' : decoding
                    , 'pageNo' : '1'
                    , 'numOfRows' : '10000'
                    , 'LAWD_CD' : sgg_code
                    ,'DEAL_YMD' : deal_ymd
                    }
            response = requests.get(url, headers=headers, params=params).text

            xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
            rows = xmlobj.find_all('item')

            # tmp data 적재
            data = []
            for row in rows:
                row_dict = {}
                for tag in row.find_all():
                    text = tag.text.strip() if tag.text.strip() else None
                    row_dict[tag.name] = text
                data.append(row_dict)
            df = pd.DataFrame(data)
            df['sggnm'] = pd.to_numeric(df['sggCd']).map(sgg_dict)
            df['거래연월']=deal_ymd
            df_saved = pd.concat([df_saved,df],ignore_index=True)
    df_saved['거래연분기'] = df_saved['거래연월'].apply(fn_quarter)
    df_saved = df_saved.drop_duplicates().reset_index(drop=True)
    df_saved['excluUseAr'] = pd.to_numeric(df_saved['excluUseAr'])
    df_saved['floor'] = pd.to_numeric(df_saved['floor'])
    if opt=="buy":
        df_saved['dealAmount'] = pd.to_numeric(df_saved['dealAmount'].str.replace(",",""))
    else :
        df_saved['monthlyRent'] = pd.to_numeric(df_saved['monthlyRent'].str.replace(",",""))
        df_saved['deposit'] = pd.to_numeric(df_saved['deposit'].str.replace(",",""))

    return df_saved


In [7]:
# 함수 실행
# 2020년09월부터 2020년10월까지, 매매거래 scraping
df_cost_new = fn_ym_parsing(202009,202010, opt="buy")

# 2020년09월부터 2020년10월까지, 전월세거래 scraping
# df_rent_new = fn_ym_parsing(202009,202010, opt="rent")

In [8]:
df_cost_new

,aptDong,aptNm,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,dealDay,dealMonth,dealYear,...,floor,jibun,landLeaseholdGbn,rgstDate,sggCd,slerGbn,umdNm,sggnm,거래연월,거래연분기
0,None,경희궁의아침3단지,2004,None,None,None,165000,30,9,2020,...,7,72,N,None,11110,None,내수동,종로구,202009,2020Q3
1,None,창신쌍용2,1993,None,None,None,71500,5,9,2020,...,12,703,N,None,11110,None,창신동,종로구,202009,2020Q3
2,None,갑을아파트,1997,None,None,None,32000,29,9,2020,...,13,595,N,None,11110,None,평창동,종로구,202009,2020Q3
3,None,경희궁자이(4단지),2017,None,None,None,102000,28,9,2020,...,8,126,N,None,11110,None,교북동,종로구,202009,2020Q3
4,None,디팰리스,2020,None,None,None,321000,23,9,2020,...,17,171,N,None,11110,None,신문로2가,종로구,202009,2020Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8461,None,다성이즈빌(121-32),2017,None,None,None,47000,14,10,2020,...,7,121-32,N,None,11740,None,천호동,강동구,202010,2020Q4
8462,None,유덕휴먼빌,2003,None,None,None,72000,9,10,2020,...,4,353-40,N,None,11740,None,길동,강동구,202010,2020Q4
8463,None,중앙하이츠,1998,None,None,None,70900,8,10,2020,...,5,305,N,None,11740,None,천호동,강동구,202010,2020Q4
8464,None,청광플러스원큐브2차,2013,None,None,None,11000,8,10,2020,...,6,457,N,None,11740,None,길동,강동구,202010,2020Q4


In [ ]:
# csv로 저장
df_cost_new.to_csv("실거래매매.csv")
# df_rent_new.to_csv("실거래_전월세.csv")